# Cifar-100 test for image classification

In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Resizing, Lambda, Flatten, Dense
import pandas as pd
import numpy as np
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

/Users/sasha/jax-metal/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Download RegNetX016 with Imagenet weights (1000 classes)

In [2]:
regnet = tf.keras.applications.regnet.RegNetX016(model_name='regnetx016',
                                                include_top=True,
                                                include_preprocessing=True,
                                                weights='imagenet',
                                                input_tensor=None,
                                                input_shape=None,
                                                pooling=None,
                                                classes=1000,
                                                classifier_activation='softmax'
                                                )

In [3]:
regnet.summary()

Model: "regnetx016"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 regnetx016_prestem_rescali  (None, 224, 224, 3)          0         ['input_1[0][0]']             
 ng (Rescaling)                                                                                   
                                                                                                  
 regnetx016_stem_conv (Conv  (None, 112, 112, 32)         864       ['regnetx016_prestem_rescaling
 2D)                                                                [0][0]']                      
                                                                                         

Make all layers untrainable except for the very last convolutional layer

In [4]:
for layer in regnet.layers:
    layer.trainable = False
regnet.layers[-6].trainable  = True

Cifar-100 testing

In [5]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

In [7]:
y_train_cat = to_categorical(y_train, 1000)
y_test_cat = to_categorical(y_test, 1000)

Lambda layer for preprocessing

In [8]:
def resize(x):
    return tf.image.resize(x,size=(224,224),method='bicubic')

Modify the model

In [9]:
input_shape = (32,32,3)

In [10]:
input_layer = Input(shape=input_shape)
prep = Lambda(resize)(input_layer)
out = regnet(prep)
regnet_mod = Model(inputs=input_layer, outputs=out)

In [11]:
regnet_mod.compile(optimizer='adam',
                   loss=tf.keras.losses.CategoricalCrossentropy(),
                   metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1_categorical_accuracy')])

Try to fit it on Cifar-100 data and then evaluate (there is no hope this is gonna work ...)

In [12]:
regnet_mod.fit(X_train, y_train_cat)

1563/1563 [==============================] - 1840s 1s/step - loss: 5.4922 - top_1_categorical_accuracy: 4.0000e-05


In [13]:
regnet_mod.evaluate(X_test, y_test_cat)

313/313 [==============================] - 366s 1s/step - loss: 5.2863 - top_1_categorical_accuracy: 0.0000e+00


[5.286314010620117, 0.0]

Try the same with adding a Cerebros "add-on" network

In [17]:
INPUT_SHAPES  = [input_shape]
OUTPUT_SHAPES = [100]

Use 10k-15k random samples from Cifar-100 to speed up the process

In [18]:
num_samples = 15_000
rng = np.random.default_rng()
ind = rng.permutation(X_train.shape[0])[:num_samples]

In [19]:
training_x   = [tf.constant(X_train[ind,:,:,:])]
y_train_cat  = to_categorical(y_train[ind], 100)
train_labels = [tf.constant(y_train_cat)]

In [20]:
regnet = tf.keras.applications.regnet.RegNetX016(model_name='regnetx016',
                                                include_top=True,
                                                include_preprocessing=True,
                                                weights='imagenet',
                                                input_tensor=None,
                                                input_shape=None,
                                                pooling=None,
                                                classes=1000,
                                                classifier_activation='softmax'
                                                )

In [21]:
for layer in regnet.layers:
    layer.trainable = False
regnet.layers[-6].trainable  = True

In [22]:
regnet_io = Model(inputs=regnet.layers[0].input,
                  outputs=regnet.layers[-2].output)

In [23]:
input_layer = Input(shape=input_shape)
prep = Lambda(resize)(input_layer)
out = Flatten()(regnet_io(prep))
base_mod = Model(inputs=input_layer, outputs=out)

In [24]:
activation = 'relu'
predecessor_level_connection_affinity_factor_first = 2.0
predecessor_level_connection_affinity_factor_main = 0.97
max_consecutive_lateral_connections = 5
p_lateral_connection = 0.97
num_lateral_connection_tries_per_unit = 2
learning_rate = 0.001
epochs = 10  # [1, 100]
batch_size = 20
maximum_levels = 4  # [3,7]
maximum_units_per_level = 7  # [2,10]
maximum_neurons_per_unit = 4  # [2,20]

In [25]:
# Final training task
TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
#
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test_cifar100_regnet'
#
meta_trial_number = 42
#
cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.2,
    direction='maximize',
    metric_to_rank_by="val_top_1_categorical_accuracy",
    minimum_levels=2,
    maximum_levels=maximum_levels,
    minimum_units_per_level=1,
    maximum_units_per_level=maximum_units_per_level,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=maximum_neurons_per_unit,
    activation=activation,
    final_activation='softmax',
    number_of_architecture_moities_to_try=3,
    number_of_tries_per_architecture_moity=2,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.TopKCategoricalAccuracy(
                k=1, name='top_1_categorical_accuracy')
             ],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number,
    base_models=[base_mod])

In [26]:
%%time
result = cerebros_automl.run_random_search()

SimpleCerebrosRandomSearch.input_shapes: [(32, 32, 3)]
nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'2': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'100': <class 'cerebros.units.units.FinalDenseUnit'>}]
2
Trying to create Final level 2
Trying to create level 2
We think level final level 2's predecessors are: [0, 1]
levels:
[0, 1, 2]
{'0': 'InputUnitModule'}
InputLevel.input_shapes [(32, 32, 3)]
{'2': <class 'cerebros.units.units.DenseUnit'>}
{'3': <class 'cerebros.units.units.DenseUnit'>}
Debug: I am 2 selecting 1
debug: meta_level_number
debug: meta_level_number
debug: meta_level_number
Setting levels_unmaterialized[0] level_number 0 to have first successor: levels_unmaterialized[:1], having level_numbers of [1, 2]
Setting levels_unmaterialized[1] level_number 1 to have first successor: level

inputs
KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'), name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'")

outputs
KerasTensor(type_spec=TensorSpec(shape=(None, 100), dtype=tf.float32, name=None), name='NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000002_tr_0_FinalDenseUnit_0000000000000002_tr_0_0_dns_/Softmax:0', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000002_tr_0_FinalDenseUnit_0000000000000002_tr_0_0_dns_'")
Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
_____________________________________________________________________

INFO:tensorflow:Assets written to: 2023_10_22_12_36_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  1.787359                    0.545333  1.481700   
1  0.888277                    0.739500  1.600415   
2  0.610960                    0.815417  1.729539   
3  0.402615                    0.872667  1.837100   
4  0.310563                    0.903583  1.864029   
5  0.248507                    0.920333  2.033121   
6  0.224195                    0.927250  2.255160   
7  0.192277                    0.937833  2.262132   
8  0.192665                    0.936833  2.199002   
9  0.173852                    0.941083  2.290837   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.612667             0                0   
1                        0.619000             0                0   
2                        0.623333             0                0   
3                        0.624333             0                0   
4                        0.634667             0   

{'2': <class 'cerebros.units.units.DenseUnit'>}
{'3': <class 'cerebros.units.units.DenseUnit'>}
debug: meta_level_number
debug: meta_level_number
Debug: successor_connectivity_errors_2d []
Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>]
materialized_predecessor_units [<KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=f

...

INFO:tensorflow:Assets written to: 2023_10_22_12_36_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  0.730148                    0.787067  1.221453   
1  0.046418                    0.993667  1.295566   
2  0.016036                    0.998667  1.332982   
3  0.022846                    0.994833  1.658705   
4  0.059721                    0.984250  1.983496   
5  0.067825                    0.977500  2.032192   
6  0.037567                    0.990000  2.227859   
7  0.048914                    0.986333  2.372792   
8  0.049732                    0.985083  2.342876   
9  0.050452                    0.983917  2.662754   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.697000             0                0   
1                        0.703667             0                0   
2                        0.708333             0                0   
3                        0.679000             0                0   
4                        0.664667             0   

{'2': <class 'cerebros.units.units.DenseUnit'>}
{'3': <class 'cerebros.units.units.DenseUnit'>}
debug: meta_level_number
debug: meta_level_number
Debug: successor_connectivity_errors_2d []
Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>]
materialized_predecessor_units [<KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 912) dtype=f

INFO:tensorflow:Assets written to: 2023_10_22_12_36_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000001/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  0.685352                    0.795867  1.196145   
1  0.033869                    0.997583  1.249168   
2  0.013692                    0.999167  1.278499   
3  0.013782                    0.997667  1.482887   
4  0.037496                    0.990167  1.902371   
5  0.061280                    0.981667  2.089307   
6  0.058171                    0.984000  2.255635   
7  0.040618                    0.987667  2.316699   
8  0.038963                    0.987917  2.524557   
9  0.033940                    0.989750  2.587205   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.694333             0                1   
1                        0.709667             0                1   
2                        0.720333             0                1   
3                        0.701667             0                1   
4                        0.647333             0   

In [27]:
print(f'Best accuracy achieved is {result}')
print(f'top-1 categorical accuracy')

Best accuracy achieved is 0.7203333377838135
top-1 categorical accuracy


Evaluating the best model found

In [28]:
best_model_found = cerebros_automl.get_best_model()

In [29]:
#
eval_loss = tf.keras.losses.CategoricalCrossentropy()
#
eval_metrics =\
[tf.keras.metrics.TopKCategoricalAccuracy(k=1,\
            name='eval_top_1_categorical_accuracy'),
 tf.keras.metrics.TopKCategoricalAccuracy(k=5,\
            name='eval_top_5_categorical_accuracy')
]

In [30]:
best_model_found.compile(loss=eval_loss, metrics=eval_metrics)
best_model_found.summary()

Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 NeuralNetworkFuture_000000  [(None, 32, 32, 3)]          0         []                            
 0000000nan_tr_0_InputLevel                                                                       
 _0000000000000000_tr_0_Inp                                                                       
 utUnit_0000000000000000_tr                                                                       
 _0_0_inp (InputLayer)                                                                            
                                                                                                  
 model_2 (Functional)        (None, 912)                  8320640   ['NeuralNetworkFuture_00000000
                                          

In [31]:
print("Evaluating best model found ...")
print("Loss | Top-1 accuracy | Top-5 accuracy")
y_test_cat = to_categorical(y_test, 100)
best_model_found.evaluate(X_test, y_test_cat)

Evaluating best model found ...
Loss | Top-1 accuracy | Top-5 accuracy
313/313 [==============================] - 366s 1s/step - loss: 2.6348 - eval_top_1_categorical_accuracy: 0.6492 - eval_top_5_categorical_accuracy: 0.8770


[2.63478422164917, 0.6492000222206116, 0.8769999742507935]